In [1]:
from pystac_client import Client
from pystac import Item
from odc.stac import load

In [16]:
# latlon_min = (-18.06231, 177.71210)
# latlon_max = (-17.54912, 178.25661)

# bbox = [latlon_min[1], latlon_min[0], latlon_max[1], latlon_max[0]]

# bbox over viti levu
bbox = [179.8, -18.06231, 180, -17.54912]

In [11]:
catalog = "https://stac.staging.digitalearthpacific.io/"
client = Client.open(catalog)

In [ ]:
items = client.search(collections="dep_s2_geomad", bbox=bbox).item_collection()

print(f"Found {len(items)} items")

In [4]:
items = [Item.from_file("https://dep-public-data.s3.us-west-2.amazonaws.com/dep_s2_geomad/0-3-3/070/012/2023/dep_s2_geomad_070_012_2023.stac-item.json")]

In [ ]:
data = load(items, bbox=bbox, chunks={"x": 2048, "y": 2048})
data

In [ ]:
bands = ["bcmad", "smad", "emad", "count"]

data.squeeze().emad.plot.imshow(robust=True)

In [ ]:
# bins between 500 and 2000
data.emad.plot.hist(bins=100, range=(0, 2000))

In [ ]:
data["count"].odc.explore()

In [ ]:
print(data.emad.compute().quantile([0.05, 0.95]).values)

In [6]:
import rasterio

with rasterio.Env(AWS_NO_SIGN_REQUEST="YES"):
    loaded = data[["smad", "bcmad", "emad"]].compute()

In [ ]:
for var in ["smad", "bcmad", "emad"]:
    # print min, max, and 10/90 percentiles
    print(
        f"{var}: {loaded[var].min().values}, {loaded[var].max().values}, {loaded[var].quantile([0.05, 0.95]).values}"
    )

In [ ]:
loaded.emad.plot.hist(bins=100)

In [ ]:
import folium

m = folium.Map(location=loaded.odc.geobox.extent.centroid.to_crs("epsg:4326").coords[0][::-1], zoom_start=10)

# loaded.odc.to_rgba(vmin=1000, vmax=3000).odc.add_to(m)

for band in ["smad", "emad", "bcmad"]:
    loaded[band].odc.add_to(m, robust=True)

folium.LayerControl().add_to(m)

m

In [ ]:
# m.save("index.html")